## Organize citation data to be able to perform clustering analysis.  Additionally, explore citation data to be able to ensure I am including all relevant features.

### Data Source
On a weekly basis the City of Scottsdale Police Department publishes a [dataset](http://data.scottsdaleaz.gov/dataset/police-citations)* which includes one prior years worth citations.  These citations are largely made up of vehicle related infractions such as speeding.  


I downloaded the data in September 2018, September 2019, and January 2020.  In raw format the combined dataset includes just over 56K rows and 22 columns.  For privacy issues, the City of Scottsdale does not release citation data for anyone under 18.  

<br/>

*Contains information from the City of Scottsdale Open Database Portal, which is made available here under the [Open Database License](http://www.scottsdaleaz.gov/AssetFactory.aspx?did=69351).


## DATA PREPROCESSING

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import datetime
import glob
pd.set_option('display.max_columns', None)
sns.set(style='darkgrid')
# plt.style.use('bmh')
plt.rcParams['patch.edgecolor'] = 'none'
import jenkspy

import hdbscan
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.cluster import AgglomerativeClustering

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

In [4]:
# read in csv files
files = glob.glob('data/primary_data/*.csv')

citations = pd.concat([pd.read_csv(f) \
                      for f in files], sort=True)

In [5]:
citations.head()

,Arizona Statute Code,Beat,Charge Description,Citation #,Citation Date,Citation Time,Citation Type Code,Citation Type Description,Cited Person,Cited Person Age,Cited Person Ethnicity,Cited Person Race,Cited Person Sex,City,District,Officer Badge #,State,Street,Street Number,Zip,Zone,tcmainid
0,28-701A,12.0,Speed Greater Than R&P or Posted,8096915,07/10/2019,747,CV,Civil Traffic,"Hansen, Christopher",19,N,W,M,Scottsdale,D3,1335,AZ,E Shea Bl,6xxx,NaN,3003,662799
1,28-701A,12.0,Speed Greater Than R&P or Posted,8096916,07/10/2019,800,CV,Civil Traffic,"Koch, JOshua",36,N,W,M,Scottsdale,D3,1335,AZ,E Shea Bl,6xxx,NaN,3003,662801
2,28-701A,7.0,Speed Greater Than R&P or Posted,8096917,07/10/2019,853,CV,Civil Traffic,"Dahl, Renee",61,N,W,F,Scottsdale,D2,1335,AZ,N Goldwater Bl,4xxx,NaN,1102,662802
3,28-701A,7.0,Speed Greater Than R&P or Posted,8096918,07/11/2019,843,CV,Civil Traffic,"Lu, Hung",59,N,A,M,Scottsdale,D2,1335,AZ,N Goldwater Bl,4xxx,NaN,1102,662804
4,28-701A,19.0,Speed Greater Than R&P or Posted,8095091,07/06/2019,2215,CV,Civil Traffic,"Noller-Johnson, Aarron",23,U,W,M,Scottsdale,D4,1450,AZ,N Pima Rd,2xxx,NaN,3801,662805


In [6]:
# convert column labels to lower case and remove spaces from column names
citations.columns = citations.columns.str.replace(' ', '_')
citations.columns = map(str.lower, citations.columns)

In [7]:
# remove duplicate citations due to overlap in data being downloaded
citations.drop_duplicates('citation_#', inplace=True)

In [8]:
# change data type of age from object to number
citations.cited_person_age = pd.to_numeric(citations.cited_person_age, errors='coerce')

In [9]:
# convert citation date to datetime format
citations['citation_date'] = pd.to_datetime(citations['citation_date'])

# create month column
citations['month'] = citations['citation_date'].dt.month

# create day of week column
citations['day_of_week'] = citations['citation_date'].dt.dayofweek

# create day of week column
citations['week_of_year'] = citations['citation_date'].dt.weekofyear

# extract the hour of the day from the citation time column
citations['hour_of_day'] = citations['citation_time'].apply(lambda x: str(x)[-4:-2])

# replace blanks with 0
# this happened for citations that took place during the hour of 12am
citations.loc[citations.hour_of_day == '', 'hour_of_day'] = 0
# change data type of hour from object to number
citations.hour_of_day = pd.to_numeric(citations.hour_of_day)

In [10]:
# one citation has no age
# drop citations where age isn't provided

citations.dropna(subset=['cited_person_age'], inplace=True)

In [11]:
citations.cited_person_sex.value_counts()

M    35199
F    21220
U       14
Name: cited_person_sex, dtype: int64

In [12]:
# 14 citations have no gender
# drop citations where gender isn't binary
citations.drop(citations[citations.cited_person_sex == 'U'].index, inplace=True)

In [13]:
# verify non-binary genders have been dropped
citations.cited_person_sex.value_counts()

M    35184
F    21216
Name: cited_person_sex, dtype: int64

In [14]:
# how many rows and columns are in the dataset?
citations.shape

(56403, 26)

## DEFINE FEATURES TO PERFORM ANALYSIS ON

In [15]:
# top 10 citations
citations.charge_description.value_counts().head(10)

Speed Greater Than R&P or Posted                 13343
Expired/No Arizona Registration                   4597
Suspended/Revoked D.L.                            3382
DUI-Impaired to Slightest Degree                  2742
Failure to Control Speed to Avoid A Collision     2670
No Proof of Insurance                             2515
Drug Paraphernalia-Possess/Use                    2506
Fail to Obey Traffic Control Device               1870
Fail to Stop For Red Signal                       1532
Fail to Stop at Stop Sign                         1469
Name: charge_description, dtype: int64

In [16]:
# top 10 citations makes up over half of all citations
citations.charge_description.value_counts().head(10).sum()

36626

In [31]:
top_citations = list(citations.charge_description.value_counts().head(10).index)

In [20]:
top_locations = list(citations.street.value_counts().head(50).index)

In [22]:
top_locations[0:20]

['N Scottsdale Rd',
 'E Camelback Rd',
 'E Indian School Rd',
 'N Hayden Rd',
 'E Shea Bl',
 'E McDowell Rd',
 'E Thomas Rd',
 'N Pima Rd',
 'N Saddlebag Tr',
 'E Indian Bend Rd',
 'N Miller Rd',
 'E Chaparral Rd',
 'E Via Linda',
 'E Cactus Rd',
 'E Indian Pz',
 'N Scottsdale Rd / E Thomas Rd',
 'N 64th St / E Cholla St',
 'N 64th St',
 'E Frank Lloyd Wright Bl / L101',
 'E Osborn Rd']

In [17]:
# create feature for specific location each citation took place at
# in case I want a higher level of detail than provided simply by streets
citations['place_id'] = citations.street + ' - ' + citations.street_number

In [18]:
# create feature for higher location granularity than simply street
top_locations_place_id = list(citations.place_id.value_counts().head(50).index)

In [32]:
# subset full citation dataset with only the top 20 citations by count and
# the top 50 citation locations
citations = citations[citations.charge_description.isin(top_citations) &
                     citations.street.isin(top_locations)]

In [33]:
# how many citation meet the above criteria?
citations.shape

(18247, 27)

In [34]:
# segment age into buckets
citations['age_breaks'] = pd.qcut(citations.cited_person_age, q=5)

In [35]:
citations.age_breaks.value_counts().sort_index()

(17.999, 24.0]    4431
(24.0, 29.0]      3132
(29.0, 37.0]      3395
(37.0, 50.0]      3935
(50.0, 97.0]      3354
Name: age_breaks, dtype: int64

In [36]:
citations.head()

,arizona_statute_code,beat,charge_description,citation_#,citation_date,citation_time,citation_type_code,citation_type_description,cited_person,cited_person_age,cited_person_ethnicity,cited_person_race,cited_person_sex,city,district,officer_badge_#,state,street,street_number,zip,zone,tcmainid,month,day_of_week,week_of_year,hour_of_day,place_id,age_breaks
0,28-701A,12.0,Speed Greater Than R&P or Posted,8096915,2019-07-10,747,CV,Civil Traffic,"Hansen, Christopher",19.0,N,W,M,Scottsdale,D3,1335,AZ,E Shea Bl,6xxx,NaN,3003,662799,7,2,28,7,E Shea Bl - 6xxx,"(17.999, 24.0]"
1,28-701A,12.0,Speed Greater Than R&P or Posted,8096916,2019-07-10,800,CV,Civil Traffic,"Koch, JOshua",36.0,N,W,M,Scottsdale,D3,1335,AZ,E Shea Bl,6xxx,NaN,3003,662801,7,2,28,8,E Shea Bl - 6xxx,"(29.0, 37.0]"
2,28-701A,7.0,Speed Greater Than R&P or Posted,8096917,2019-07-10,853,CV,Civil Traffic,"Dahl, Renee",61.0,N,W,F,Scottsdale,D2,1335,AZ,N Goldwater Bl,4xxx,NaN,1102,662802,7,2,28,8,N Goldwater Bl - 4xxx,"(50.0, 97.0]"
3,28-701A,7.0,Speed Greater Than R&P or Posted,8096918,2019-07-11,843,CV,Civil Traffic,"Lu, Hung",59.0,N,A,M,Scottsdale,D2,1335,AZ,N Goldwater Bl,4xxx,NaN,1102,662804,7,3,28,8,N Goldwater Bl - 4xxx,"(50.0, 97.0]"
4,28-701A,19.0,Speed Greater Than R&P or Posted,8095091,2019-07-06,2215,CV,Civil Traffic,"Noller-Johnson, Aarron",23.0,U,W,M,Scottsdale,D4,1450,AZ,N Pima Rd,2xxx,NaN,3801,662805,7,5,27,22,N Pima Rd - 2xxx,"(17.999, 24.0]"


In [27]:
# combine features that I want to use for modeling
pd.concat([citations[['beat',
                    'cited_person_age',
                    'citation_time',
                    'charge_description']], 
             citations.iloc[:,-5:]], axis=1)

,beat,cited_person_age,citation_time,charge_description,month,day_of_week,week_of_year,hour_of_day,place_id
0,12.0,19.0,747,Speed Greater Than R&P or Posted,7,2,28,7,E Shea Bl - 6xxx
1,12.0,36.0,800,Speed Greater Than R&P or Posted,7,2,28,8,E Shea Bl - 6xxx
2,7.0,61.0,853,Speed Greater Than R&P or Posted,7,2,28,8,N Goldwater Bl - 4xxx
3,7.0,59.0,843,Speed Greater Than R&P or Posted,7,3,28,8,N Goldwater Bl - 4xxx
4,19.0,23.0,2215,Speed Greater Than R&P or Posted,7,5,27,22,N Pima Rd - 2xxx
...,...,...,...,...,...,...,...,...,...
22420,16.0,20.0,1224,Speed Greater Than R&P or Posted,9,6,39,12,E Frank Lloyd Wright Bl / L101 - xxx
22421,6.0,36.0,5,Urinate/Defecate In Public,9,6,39,0,N Miller Rd - 4xxx
22422,3.0,22.0,1616,Failure to Control Speed to Avoid A Collision,10,1,40,16,E Thomas Rd - 7xxx
22423,12.0,20.0,1722,Fail to Drive In A Single Lane,10,2,40,17,E Cactus Rd - 7xxx


In [29]:
# combine features that I want to use for modeling

pd.concat([citations[['beat',
                    'cited_person_age',
                    'citation_time',
                    'charge_description']], 
             citations.iloc[:,-5:]], axis=1).to_csv('data/clustering_features.csv', index=False)

## EDA

### "Beat" seems to describe patrol locations within the city, with the lowest beat numbers being the farthest south, and the highest beat numbers being in the north.

In [104]:
citations.beat.value_counts().sort_index()

1.0      868
2.0      830
3.0     1629
4.0      959
5.0      820
6.0     1682
7.0     1521
8.0      550
9.0      621
10.0    1134
11.0     459
12.0    1381
13.0     594
14.0     683
15.0    1068
16.0     559
17.0     578
18.0     348
19.0    1398
20.0     382
99.0     126
Name: beat, dtype: int64

In [102]:
citations.loc[citations.beat == 1, 'street'].value_counts()

E McDowell Rd                  353
N Hayden Rd / E McDowell Rd    186
E McKellips Rd                 133
N Hayden Rd                     67
N Miller Rd                     61
N Scottsdale Rd                 60
N Granite Reef Rd                4
N 68th St                        4
Name: street, dtype: int64

In [108]:
citations.loc[citations.beat == 15, 'street'].value_counts()

E Redfield Rd                                297
E Frank Lloyd Wright Bl                      223
E Frank Lloyd Wright Bl / N Hayden Rd        175
E Frank Lloyd Wright Bl / N Scottsdale Rd    168
N Scottsdale Rd                              157
E Cactus Rd                                   32
N Hayden Rd                                   15
N 75th St                                      1
Name: street, dtype: int64

In [106]:
citations.loc[citations.beat == 19, 'street'].value_counts()

N Hayden Rd                  435
N Pima Rd                    424
N Scottsdale Rd              358
N Hayden Rd / E Legacy Bl    173
N Miller Rd                    8
Name: street, dtype: int64